In [138]:
## remember to run conn_cocolab from the terminal before running cells in this notebook!

import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64

In [139]:
# directory & file hierarchy
iterationName = 'test'
exp_path = 'museumdraw'
analysis_dir = os.getcwd()
sketch_dir = os.path.join(analysis_dir,'sketches')
if not os.path.exists(sketch_dir):
    os.makedirs(sketch_dir)
exp_dir = os.path.abspath(os.path.join(os.getcwd(),'../..','experiments'))

In [140]:
# set vars 
auth = pd.read_csv('../auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['kiddraw']
coll = db['E1b']
collPilot = db['E1']

In [141]:
coll


Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), u'kiddraw'), u'E1b')

In [142]:
collPilot

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), u'kiddraw'), u'E1')

In [143]:
final_images = coll.find({'dataType':'finalImage'}).sort('time')

imsize = 224
for rec in final_images:
    imgData = rec['imgData'];
    filestr = base64.b64decode(imgData)
    if 'age' in rec.keys():
        if rec['trialNum']>0:
            print('got age and sketches')
            fname = os.path.join(sketch_dir,'{}_sketch_Age_{}_{}.png'.format(rec['category'],rec['age'],rec['trialNum']))
            with open(fname, "wb") as fh:
                fh.write(imgData.decode('base64'))  

got age and sketches
got age and sketches
got age and sketches
got age and sketches
got age and sketches
got age and sketches


In [144]:
final_images = collPilot.find({'dataType':'finalImage'}).sort('time')
imsize = 224
for rec in final_images:
    imgData = rec['imgData'];
    filestr = base64.b64decode(imgData)
    if rec['trialNum']>0:
        print('got sketches')
        fname = os.path.join(sketch_dir,'{}_sketch_{}.png'.format(rec['category'],rec['trialNum']))
        with open(fname, "wb") as fh:
            fh.write(imgData.decode('base64'))

got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches
got sketches

[u'category',
 u'dataType',
 u'trialNum',
 u'sessionId',
 u'time',
 u'date',
 u'imgData',
 u'_id']

In [53]:
imsize

224